In [55]:
## IMPORTACIÓN GENERAL DE LIBRERIAS.
import re 
import sys
import json
import nltk
import geocoder
import requests
import warnings
import descartes

import numpy as np
import pandas as pd
import seaborn as sns
import networkx as nx
import datetime as DT
#import geopandas as gpd
import matplotlib.pyplot as plt

# SKLEARN.
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

# RANDOM FOREST.
from urllib.request import urlopen
#from shapely.geometry import Point, Polygon
from sklearn.ensemble import RandomForestRegressor

# XGBOOST.
from xgboost import XGBClassifier
import xgboost as xgb

In [56]:
#NLTK: https://www.nltk.org
from langdetect import detect
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from nltk.tokenize import TweetTokenizer
from nltk.stem.wordnet import WordNetLemmatizer

#WORDCLOUD
from PIL import Image
from wordcloud import WordCloud

# CONFIGURACIÓN.
%matplotlib inline
plt.style.use('default')
pd.options.display.float_format = '{:20,.2f}'.format
warnings.filterwarnings('ignore')
sns.set(style="whitegrid") 
plt.rcParams['figure.figsize'] = (15, 10)
np.set_printoptions(threshold=sys.maxsize)

In [57]:
#Importacion del archivo CSV de fuente
#https://www.kaggle.com/c/nlp-getting-started
original_train = pd.read_csv('data/train.csv')
original_test = pd.read_csv('data/test.csv')
original_sample_submission = pd.read_csv('data/sample_submission.csv')

In [58]:
#PASAMOS TODO A MINÚSCULAS (TRAIN).
original_train['text'] = original_train['text'].str.lower()
original_train['location'] = original_train['location'].str.lower()
original_train['keyword'] = original_train['keyword'].str.lower()
#PASAMOS TODO A MINÚSCULAS (TEST).
original_test['text'] = original_test['text'].str.lower()
original_test['location'] = original_test['location'].str.lower()
original_test['keyword'] = original_test['keyword'].str.lower()

In [59]:
#LONGITUD DEL TWEET Y CANTIDAD DE PALABRAS (TRAIN).
original_train['length'] = original_train['text'].str.len()
original_train['totalwords'] = original_train['text'].str.split().str.len()
#LONGITUD DEL TWEET Y CANTIDAD DE PALABRAS (TEST).
original_test['length'] = original_test['text'].str.len()
original_test['totalwords'] = original_test['text'].str.split().str.len()

In [60]:
original_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          7613 non-null   int64 
 1   keyword     7552 non-null   object
 2   location    5080 non-null   object
 3   text        7613 non-null   object
 4   target      7613 non-null   int64 
 5   length      7613 non-null   int64 
 6   totalwords  7613 non-null   int64 
dtypes: int64(4), object(3)
memory usage: 416.5+ KB


In [61]:
def clean_tweet(words):
    result = []
    for word in words:
        stripped_word = word.strip()
        if ((stripped_word.isalnum() == True) and (not(stripped_word.isdigit()))): # or (stripped_word[0] == '#') or (stripped_word[0] == '@')):       
            result.append(stripped_word)
    
    return result
 
#Separamos las plabras de los tweets, usamos el tokenizador de twitter de nltk
tt = TweetTokenizer()
original_train['text_tokenized'] = original_train['text'].apply(tt.tokenize)
original_train['text_tokenized'] = original_train['text_tokenized'].apply(clean_tweet)

original_test['text_tokenized'] = original_test['text'].apply(tt.tokenize)
original_test['text_tokenized'] = original_test['text_tokenized'].apply(clean_tweet)


In [62]:
original_train.head()

,id,keyword,location,text,target,length,totalwords,text_tokenized
0,1,NaN,NaN,our deeds are the reason of this #earthquake m...,1,69,13,"[our, deeds, are, the, reason, of, this, may, ..."
1,4,NaN,NaN,forest fire near la ronge sask. canada,1,38,7,"[forest, fire, near, la, ronge, sask, canada]"
2,5,NaN,NaN,all residents asked to 'shelter in place' are ...,1,133,22,"[all, residents, asked, to, shelter, in, place..."
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,65,8,"[people, receive, evacuation, orders, in, cali..."
4,7,NaN,NaN,just got sent this photo from ruby #alaska as ...,1,88,16,"[just, got, sent, this, photo, from, ruby, as,..."


In [63]:
original_train.head()

,id,keyword,location,text,target,length,totalwords,text_tokenized
0,1,NaN,NaN,our deeds are the reason of this #earthquake m...,1,69,13,"[our, deeds, are, the, reason, of, this, may, ..."
1,4,NaN,NaN,forest fire near la ronge sask. canada,1,38,7,"[forest, fire, near, la, ronge, sask, canada]"
2,5,NaN,NaN,all residents asked to 'shelter in place' are ...,1,133,22,"[all, residents, asked, to, shelter, in, place..."
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,65,8,"[people, receive, evacuation, orders, in, cali..."
4,7,NaN,NaN,just got sent this photo from ruby #alaska as ...,1,88,16,"[just, got, sent, this, photo, from, ruby, as,..."


In [65]:
tweets_words_filtered = original_train['text_tokenized']
#Obtenemos los lemas
lem = WordNetLemmatizer()
tweets_words_lemmatized=[]
for word in tweets_words_filtered:
    tweets_words_lemmatized.append(lem.lemmatize(word,"v"))

TypeError: unhashable type: 'list'

In [67]:
type(tweets_words_filtered)

pandas.core.series.Series

In [49]:
#MARCAMOS QUIENES USAN UBICACIÓN Y PALABRAS CLAVES (TRAIN).
original_train['conUbicacion'] = 0
original_train['conKeyword'] = 0
original_train.loc[original_train['location'] != 'vacio', ['conUbicacion']] = 1
original_train.loc[original_train['keyword'] != 'vacio', ['conKeyword']] = 1
#MARCAMOS QUIENES USAN UBICACIÓN Y PALABRAS CLAVES (TEST).
original_test['conUbicacion'] = 0
original_test['conKeyword'] = 0
original_test.loc[original_test['location'] != 'vacio', ['conUbicacion']] = 1
original_test.loc[original_test['keyword'] != 'vacio', ['conKeyword']] = 1

In [50]:
#HOT ENCODING PARA KEYWORD (TRAIN).
dummies = pd.get_dummies(original_train['keyword'], drop_first=False)
original_train = pd.concat([original_train, dummies], axis=1)
original_train.drop('keyword', 1, inplace = True)
#HOT ENCODING PARA KEYWORD (TEST).
dummies = pd.get_dummies(original_test['keyword'], drop_first=False)
original_test = pd.concat([original_test, dummies], axis=1)
original_test.drop('keyword', 1, inplace = True)

In [51]:
#ELIMINAMOS LAS COLUMNAS QUE NO SON NUMÉRICAS (TRAIN).
original_train.drop('text tokenized', 1, inplace = True)
original_train.drop('location', 1, inplace = True)
#ELIMINAMOS LAS COLUMNAS QUE NO SON NUMÉRICAS (TEST).
original_test.drop('text tokenized', 1, inplace = True)
original_test.drop('location', 1, inplace = True)

In [52]:
original_train.head()

,id,text,target,length,totalwords,conUbicacion,conKeyword,ablaze,accident,aftershock,...,weapons,whirlwind,wild%20fires,wildfire,windstorm,wounded,wounds,wreck,wreckage,wrecked
0,1,our deeds are the reason of this #earthquake m...,1,69,13,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,4,forest fire near la ronge sask. canada,1,38,7,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,5,all residents asked to 'shelter in place' are ...,1,133,22,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,6,"13,000 people receive #wildfires evacuation or...",1,65,8,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,7,just got sent this photo from ruby #alaska as ...,1,88,16,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [36]:
original_train.to_csv('data/processed/train_to_keras.csv',index=False)
original_test.to_csv('data/processed/test_to_keras.csv',index=False)